<a href="https://colab.research.google.com/github/billiahkerubo1/Data-Science-Portfolio/blob/main/NLP_Amazon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import pandas as pd #data manipulation
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

y=pd.read_csv('/content/Reviews.csv')
y.head()



,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


Rid Of Very Unhelpful Reviews & Narrow It To First 5000 Due To Data Size

In [ ]:
train_set = y.loc[(y['Score'] > 2) & (y['Id'] < 15000)]

Bring in packages to get rid of Stopwords. This helps get rid of junk words that aren't meaningful

In [ ]:
from nltk.corpus import stopwords
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger') 
import string
from nltk import word_tokenize, pos_tag

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


Loop to strip out nouns

In [ ]:
def nouns(text):
    '''Given a string of text, tokenize the text and pull out only the nouns.'''
    is_noun = lambda pos: pos[:2] == 'NN'
    tokenized = word_tokenize(text)
    all_nouns = [word for (word, pos) in pos_tag(tokenized) if is_noun(pos)] 
    return ' '.join(all_nouns)

Now use the loop to strip out nouns from the Text column

In [ ]:
data_nouns = pd.DataFrame(train_set.Text.apply(nouns))

Add Stop Words to remove words that aren't useful to analysis but common

In [ ]:
from sklearn.feature_extraction import text
# Re-add the additional stop words since we are recreating the document-term matrix
add_stop_words = ['like', 'im', 'know', 'just', 'dont', 'thats', 'right', 'people',
                  'youre', 'got', 'gonna', 'time', 'think', 'yeah', 'said','br']
stop_words = text.ENGLISH_STOP_WORDS.union(add_stop_words)


Turn Data Set Into A Matrix To Look Into Reviews (you will see in the last line we are only looking at the Text field)

In [ ]:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(stop_words=stop_words)
data_cv = cv.fit_transform(data_nouns.Text)
data_dtmn = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtmn.index = data_nouns.index


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
from gensim import matutils, models
import scipy.sparse


In [ ]:
# Create the gensim corpus
corpusn = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmn.transpose()))

# Create the vocabulary dictionary
id2wordn = dict((v, k) for k, v in cv.vocabulary_.items())

Get ready to build model

In [ ]:
ldan = models.LdaModel(corpus=corpusn, num_topics=2, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.018*"treats" + 0.017*"dog" + 0.016*"food" + 0.015*"product" + 0.012*"dogs" + 0.009*"amazon" + 0.009*"treat" + 0.008*"crackers" + 0.008*"bag" + 0.007*"price"'),
 (1,
  '0.050*"coffee" + 0.020*"flavor" + 0.019*"tea" + 0.017*"cup" + 0.017*"taste" + 0.013*"chocolate" + 0.011*"product" + 0.009*"sugar" + 0.009*"cups" + 0.009*"cookies"')]

In [ ]:
ldan = models.LdaModel(corpus=corpusn, num_topics=3, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.091*"coffee" + 0.032*"cup" + 0.024*"flavor" + 0.017*"cups" + 0.017*"taste" + 0.010*"chocolate" + 0.009*"keurig" + 0.009*"roast" + 0.009*"amazon" + 0.009*"price"'),
 (1,
  '0.028*"treats" + 0.028*"dog" + 0.024*"food" + 0.020*"product" + 0.020*"tea" + 0.019*"dogs" + 0.014*"treat" + 0.013*"amazon" + 0.009*"price" + 0.008*"bag"'),
 (2,
  '0.016*"taste" + 0.015*"flavor" + 0.011*"crackers" + 0.010*"product" + 0.010*"cookies" + 0.009*"mix" + 0.009*"sugar" + 0.008*"chocolate" + 0.007*"water" + 0.007*"milk"')]